In [1]:
import pandas as pd 
import numpy as np 
from scipy.stats import entropy
import matplotlib.pyplot as plt

from engine_hms_trainer import *
from engine_hms_model import CustomModel, JobConfig, ModelConfig

import torch
from torch import nn
import torch.nn.functional as F

/home/shiyi/miniconda3/envs/kaggle/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
seed_everything(JobConfig.SEED)

ModelConfig.EPOCHS = 6
ModelConfig.USE_EEG_SPECTROGRAMS = False
ModelConfig.MODEL_BACKBONE = 'tf_efficientnet_b2'
ModelConfig.MODEL_NAME = "ENet_b2_xymask"
ModelConfig.AUGMENT = True
ModelConfig.USE_KAGGLE_SPECTROGRAMS = True
ModelConfig.USE_EEG_SPECTROGRAMS = True

ModelConfig.AUGMENTATIONS = ['xy_masking']

hms_predictor = HMSPredictor(JobConfig, ModelConfig)

****************************************************************************************************
Script Start: Sun Mar 10 00:46:23 2024
Initializing HMS Predictor...
Model Name: ENet_b2_xymask
Drop Rate: 0.15
Drop Path Rate: 0.25
Augment: True
Augmentations: ['xy_masking']
Enropy Split: 5.5
Device: cuda
Output Dir: ./outputs/
****************************************************************************************************


In [3]:
train_easy, train_hard, all_specs, all_eegs = hms_predictor.load_train_data()

print(train_easy.shape)
print(train_hard.shape)

# check if contain NaN
print(train_easy.isnull().sum().sum())
print(train_hard.isnull().sum().sum())

display(train_easy.head())
print(" ")
display(train_hard.head())

(12440, 14)
(5536, 14)
0
0


,eeg_id,spectrogram_id,min,max,patient_id,target,total_votes,entropy,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,642382,14960202,1008.0,1032.0,5955,Other,2,7.802343,0.0,0.0,0.0,0.0,0.0,1.0
1,751790,618728447,908.0,908.0,38549,GPD,1,7.802343,0.0,0.0,1.0,0.0,0.0,0.0
2,778705,52296320,0.0,0.0,40955,Other,2,7.686820,0.0,0.0,0.0,0.0,0.0,1.0
3,1629671,2036345030,0.0,160.0,37481,Seizure,51,7.619243,1.0,0.0,0.0,0.0,0.0,0.0
4,2061593,320962633,1450.0,1450.0,23828,Other,1,7.802343,0.0,0.0,0.0,0.0,0.0,1.0


,eeg_id,spectrogram_id,min,max,patient_id,target,total_votes,entropy,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,568657,789577333,0.0,16.0,20654,Other,48,3.341757,0.000000,0.000000,0.250000,0.000000,0.166667,0.583333
1,582999,1552638400,0.0,38.0,20230,LPD,154,3.550549,0.000000,0.857143,0.000000,0.071429,0.000000,0.071429
2,1895581,128369999,1138.0,1138.0,47999,Other,13,3.565051,0.076923,0.000000,0.000000,0.000000,0.076923,0.846154
3,2482631,978166025,1902.0,1944.0,20606,Other,105,1.431066,0.000000,0.000000,0.133333,0.066667,0.133333,0.666667
4,2521897,673742515,0.0,4.0,62117,Other,24,1.516203,0.000000,0.000000,0.083333,0.083333,0.333333,0.500000


In [4]:
hms_predictor.train_folds(train_easy, train_hard, all_specs, all_eegs)

Fold: 0 || Valid size 3596 
- First Stage 


Train:   0%|          | 0/622 [00:00<?, ?batch/s]

Epoch: [1][0/622]Elapsed 1.08s | Loss: 0.8220 Grad: 67731.3828 LR: 4.0000e-06
Epoch: [1][50/622]Elapsed 8.17s | Loss: 0.8346 Grad: 85699.5469 LR: 5.0647e-06
Epoch: [1][100/622]Elapsed 15.29s | Loss: 0.8281 Grad: 70401.9922 LR: 8.2116e-06
Epoch: [1][150/622]Elapsed 22.41s | Loss: 0.8233 Grad: 75559.6250 LR: 1.3301e-05
Epoch: [1][200/622]Elapsed 29.53s | Loss: 0.8153 Grad: 72208.4922 LR: 2.0107e-05
Epoch: [1][250/622]Elapsed 36.67s | Loss: 0.8045 Grad: 70698.4688 LR: 2.8328e-05
Epoch: [1][300/622]Elapsed 43.82s | Loss: 0.7909 Grad: 78792.6719 LR: 3.7599e-05
Epoch: [1][350/622]Elapsed 51.00s | Loss: 0.7736 Grad: 118346.3359 LR: 4.7509e-05
Epoch: [1][400/622]Elapsed 58.20s | Loss: 0.7534 Grad: 59210.1992 LR: 5.7619e-05
Epoch: [1][450/622]Elapsed 65.43s | Loss: 0.7316 Grad: 76243.6875 LR: 6.7479e-05
Epoch: [1][500/622]Elapsed 72.67s | Loss: 0.7124 Grad: 71753.1094 LR: 7.6652e-05
Epoch: [1][550/622]Elapsed 79.88s | Loss: 0.6960 Grad: 73817.9453 LR: 8.4732e-05
Epoch: [1][600/622]Elapsed 87.10

Valid:   0%|          | 0/225 [00:00<?, ?batch/s]

Epoch: [1][0/225]Elapsed 0.10s | Loss: 0.3521
Epoch: [1][50/225]Elapsed 4.94s | Loss: 0.5367
Epoch: [1][100/225]Elapsed 9.77s | Loss: 0.5370
Epoch: [1][150/225]Elapsed 14.60s | Loss: 0.5320
Epoch: [1][200/225]Elapsed 19.46s | Loss: 0.4840


...

Epoch: [6][0/276]Elapsed 0.10s | Loss: 0.1284 Grad: 106348.1250 LR: 7.2357e-05
Epoch: [6][50/276]Elapsed 7.53s | Loss: 0.1407 Grad: 93076.1172 LR: 6.9971e-05
Epoch: [6][100/276]Elapsed 15.01s | Loss: 0.1366 Grad: 55417.1328 LR: 6.7529e-05
Epoch: [6][150/276]Elapsed 22.49s | Loss: 0.1379 Grad: 100059.5703 LR: 6.5039e-05
Epoch: [6][200/276]Elapsed 29.95s | Loss: 0.1370 Grad: 56523.1875 LR: 6.2507e-05
Epoch: [6][250/276]Elapsed 37.41s | Loss: 0.1353 Grad: 77437.1172 LR: 5.9941e-05
Epoch: [6][275/276]Elapsed 41.20s | Loss: 0.1346 Grad: 159898.2812 LR: 5.8595e-05


Valid:   0%|          | 0/225 [00:00<?, ?batch/s]

Epoch: [6][0/225]Elapsed 0.10s | Loss: 0.3241
Epoch: [6][50/225]Elapsed 4.94s | Loss: 0.3623
Epoch: [6][100/225]Elapsed 9.78s | Loss: 0.3599
Epoch: [6][150/225]Elapsed 14.62s | Loss: 0.3651
Epoch: [6][200/225]Elapsed 19.46s | Loss: 0.3297


----------------------------------------------------------------------------------------------------
Epoch 6 - Average Train Loss: 0.1346 | Average Valid Loss: 0.3172 | Time: 63.15s
Best model found in epoch 6 | valid loss: 0.3172
Fold 4 Valid Loss: 
Easy: 0.7334 | Hard: 0.4122
Elapse: 88.86 min 


IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

In [ ]:
dataset = CustomDataset(train_easy, TARGETS, ModelConfig, all_specs, all_eegs, mode='test')

X, y = dataset[0]
print(X.shape, y.shape)

model = CustomModel(ModelConfig, num_classes=6, pretrained=True)
y_pred = model(X.unsqueeze(0))

print(y_pred.shape)

In [ ]:
from kl_divergence import score as kl_score


def calc_kl_div(p, q, criterion):
    
    p = torch.tensor(p.astype(np.float32)).unsqueeze(0)
    q = torch.tensor(q.astype(np.float32)).unsqueeze(0)
    return criterion(F.log_softmax(p, dim=1), q).item()

def calc_kaggle_score(solution, submission):
    solution = solution.to_frame().T
    solution[TARGETS] = solution[TARGETS].astype(np.float32)
    submission = submission.to_frame().T
    submission.columns = ['eeg_id'] + TARGETS
    submission[TARGETS] = submission[TARGETS].astype(np.float32)
    
    return kl_score(solution, submission, 'eeg_id')

In [ ]:
def evaluate_oof(oof_csv_path):
    oof_df = pd.read_csv(oof_csv_path)
    softmax = nn.Softmax(dim=1)
    criterion = nn.KLDivLoss(reduction="batchmean")

    oof_df["kl_loss"] = oof_df.apply(lambda row: 
        calc_kl_div(row[TARGETS_PRED].values, row[TARGETS].values, criterion), axis=1
        )

    kl_loss_all = criterion(
        F.log_softmax(torch.tensor(oof_df[TARGETS_PRED].values.astype(np.float32)), dim=1),
        torch.tensor(oof_df[TARGETS].values.astype(np.float32)),
        )

    print(f"KL Loss All: {kl_loss_all}")
    print(f"KL Loss Mean: {oof_df['kl_loss'].mean()}")

    y_pred = oof_df[TARGETS].values.astype(np.float32)
    oof_df[TARGETS_PRED] = softmax(torch.tensor(y_pred)).numpy()

    solution = oof_df[['eeg_id'] + TARGETS].copy()
    submission = oof_df[['eeg_id'] + TARGETS_PRED].copy()
    submission.columns = ['eeg_id'] + TARGETS

    kaggle_score_all = kl_score(solution, submission, 'eeg_id')
    
    oof_df['kaggle_score'] = oof_df.apply(lambda row:
        calc_kaggle_score(row[['eeg_id'] + TARGETS], row[['eeg_id'] + TARGETS_PRED]), axis=1
        )

    print(f"Kaggle Score All: {kaggle_score_all}")
    print(f"Kaggle Score Mean: {oof_df['kaggle_score'].mean()}")

    return oof_df, kl_loss_all, kaggle_score_all


In [ ]:
oof_1, kl_loss_all, kaggle_score_all = evaluate_oof(f"{JobConfig.OUTPUT_DIR}/oof_1.csv")
oof_2, kl_loss_all, kaggle_score_all = evaluate_oof(f"{JobConfig.OUTPUT_DIR}/oof_2.csv")

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(10, 10), sharex=True, sharey=True)

# rows = oof_df.iloc[-len(axes.ravel()):, :]
rows = oof_1.sample(len(axes.ravel()))

for i, (idx, row) in enumerate(rows.iterrows()):

    ax = axes.ravel()[i]
    ax.plot(row[TARGETS].values, label='True')
    ax.plot(row[TARGETS_PRED].values, label='Pred')
    ax.set_title(f"{idx} | {row['target']} | KL: {row['kl_loss']:.4f}")
    ax.set_xticks(range(6))
    ax.set_xticklabels(BRAIN_ACTIVITY)
    ax.grid(True)
    ax.legend()

fig.tight_layout()
fig.savefig(f"{JobConfig.OUTPUT_DIR}/oof_examples_1.png")
plt.show()

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(10, 10), sharex=True, sharey=True)

# rows = oof_2.iloc[5:5+len(axes.ravel()), :]
rows = oof_2.sample(len(axes.ravel()))

for i, (idx, row) in enumerate(rows.iterrows()):

    ax = axes.ravel()[i]
    y_true = row[TARGETS].values
    y_pred = row[TARGETS_PRED].values
    y_norm = (y_true - y_true.min()) / (y_true.max() - y_true.min())

    ax.plot(row[TARGETS].values, label='True')
    ax.plot(row[TARGETS_PRED].values, label='Pred')
    ax.plot(y_norm, "b:", label='True Norm')

    ax.set_title(f"{idx} | {row['target']} | KL: {row['kl_loss']:.4f}")
    ax.set_xticks(range(6))
    ax.set_xticklabels(BRAIN_ACTIVITY)
    ax.grid(True)
    ax.legend()

fig.tight_layout()
fig.savefig(f"{JobConfig.OUTPUT_DIR}/oof_examples_2.png")
plt.show()

In [ ]:
row = oof_2.loc[6]

min_pred = row[TARGETS_PRED].min()
max_pred = row[TARGETS_PRED].max()
print(min_pred, max_pred)

print(row[TARGETS_PRED])

targets_norm = (row[TARGETS] - row[TARGETS].min()) / (row[TARGETS].max() - row[TARGETS].min())

targets_norm = targets_norm / targets_norm.sum()

print(targets_norm)